In [ ]:
import gradio as gr  # Импортируем Gradio для создания веб-интерфейса
import numpy as np  # Для работы с массивами и случайными числами
import matplotlib.pyplot as plt  # Для визуализации
from matplotlib import colors  # Для настройки цветовой схемы
from matplotlib.animation import FuncAnimation, FFMpegWriter  # Для создания анимаций
import tempfile  # Для создания временных файлов
import os
from numba import jit  # Для ускорения вычислений с помощью JIT-компиляции

# --- Константы и цвета ---
NEIGHBOURHOOD = ((-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1))  # Смещения к 8 соседям клетки
EMPTY, TREE, FIRE = 0, 1, 2  # Коды состояний клетки: пусто, дерево, огонь
colors_list = [(0.2,0,0), (0,0.5,0), (1,0,0), 'orange']  # Цвета для визуализации: пусто, дерево, огонь, (зарезервировано)
cmap = colors.ListedColormap(colors_list)  # Создаём карту цветов
bounds = [0,1,2,3,4]  # Границы для нормализации цветов
norm = colors.BoundaryNorm(bounds, cmap.N)  # Нормализация для colormap

@jit(nopython=True)
def iterate_optimized(X_curr, p_growth, f_lightning, ny_dim, nx_dim):
    """
    Выполняет один шаг эволюции клеточного автомата:
    - Пустая клетка может вырасти в дерево с вероятностью p_growth.
    - Дерево может загореться, если сосед горит, или от молнии с вероятностью f_lightning.
    - Огонь исчезает (становится пусто).
    """
    X_next = np.zeros((ny_dim, nx_dim), dtype=X_curr.dtype)  # Новый массив для следующего состояния
    for r in range(1, ny_dim - 1):  # Не трогаем границы (рамку)
        for c in range(1, nx_dim - 1):
            current_state = X_curr[r, c]
            if current_state == EMPTY:
                # Если клетка пуста, с вероятностью p_growth вырастает дерево
                if np.random.random() <= p_growth:
                    X_next[r, c] = TREE
            elif current_state == TREE:
                X_next[r, c] = TREE  # По умолчанию дерево остаётся деревом
                caught_fire_from_neighbor = False
                for dr, dc in NEIGHBOURHOOD:  # Проверяем всех соседей
                    is_diagonal = (abs(dr) == abs(dc))
                    can_spread_from_this_neighbor = True
                    # Диагональные соседи реже поджигают (~42.7% шанс)
                    if is_diagonal and np.random.random() < 0.573:
                        can_spread_from_this_neighbor = False
                    if can_spread_from_this_neighbor:
                        if X_curr[r + dr, c + dc] == FIRE:
                            X_next[r, c] = FIRE  # Дерево загорается от соседа
                            caught_fire_from_neighbor = True
                            break  # Достаточно одного горящего соседа
                if not caught_fire_from_neighbor:
                    # Если не загорелось от соседа, может загореться от молнии
                    if np.random.random() <= f_lightning:
                        X_next[r, c] = FIRE
    return X_next

def create_forest_animation(seed=0, frames=100, size=200, p=0.02, f=0.0001, forest_fraction=0.2, fps=20):
    """
    Генерирует анимацию развития лесного пожара.
    - seed: случайное зерно для воспроизводимости
    - frames: количество кадров
    - size: размер сетки (size x size)
    - p: вероятность роста дерева
    - f: вероятность удара молнии
    - forest_fraction: доля леса в начале
    - fps: кадров в секунду
    Возвращает путь к временному mp4-файлу с анимацией.
    """
    np.random.seed(seed)  # Фиксируем зерно для воспроизводимости
    ny, nx = size, size  # Размер сетки
    X = np.zeros((ny, nx), dtype=np.int8)  # Изначально вся сетка пуста
    # Заполняем внутреннюю часть сетки деревьями с вероятностью forest_fraction
    interior_mask = np.random.random(size=(ny-2, nx-2)) < forest_fraction
    X[1:ny-1, 1:nx-1] = np.where(interior_mask, TREE, EMPTY).astype(np.int8)

    dpi = 100  # Разрешение для matplotlib
    figsize = (size / dpi, size / dpi)  # Размер фигуры в дюймах
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    ax.set_xticks([])  # Без осей X
    ax.set_yticks([])  # Без осей Y
    ax.set_axis_off()  # Полностью убираем оси
    plt.tight_layout(pad=0)
    im = ax.imshow(X, cmap=cmap, norm=norm, interpolation='nearest')  # Отображаем начальное состояние

    def animate(i):
        nonlocal X
        X = iterate_optimized(X, p, f, ny, nx)  # Следующий шаг автомата
        im.set_data(X)  # Обновляем изображение
        return [im]

    writer = FFMpegWriter(fps=fps, bitrate=int(size*size*fps*0.08))  # Настраиваем видео-энкодер
    tmp_file = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False)  # Временный файл для видео
    ani = FuncAnimation(fig, animate, frames=frames, interval=1000/fps, blit=True)  # Создаём анимацию
    ani.save(tmp_file.name, writer=writer)  # Сохраняем анимацию в файл
    plt.close(fig)  # Закрываем фигуру, чтобы не засорять память
    return tmp_file.name  # Возвращаем путь к видео

def gradio_grid(seed, grid_size, size, frames, p, f, forest_fraction):
    """
    Генерирует сетку (grid_size x grid_size) независимых анимаций лесного пожара.
    Для каждой ячейки используется свой seed.
    Возвращает список (путь к видео, подпись).
    """
    video_items = []
    for i in range(grid_size):
        for j in range(grid_size):
            cell_seed = seed + i * grid_size + j  # Для каждой ячейки свой seed
            video_path = create_forest_animation(
                seed=cell_seed, frames=frames, size=size, p=p, f=f, forest_fraction=forest_fraction
            )
            label = f"Seed: {cell_seed} ({i},{j})"  # Подпись для видео
            video_items.append((video_path, label))
    return video_items

# Интерфейс Gradio для запуска симуляции и отображения сетки анимаций
gr.Interface(
    fn=gradio_grid,  # Основная функция, вызываемая при запуске
    inputs=[
        gr.Number(label="Random Seed", value=0),  # Начальное зерно генератора случайных чисел
        gr.Slider(1, 10, value=1, step=1, label="Grid Size GR (GRxGR)"),  # Размер сетки анимаций
        gr.Slider(50, 1500, value=100, step=10, label="Forest Size (NxN)"),  # Размер одной симуляции
        gr.Slider(10, 2000, value=50, step=1, label="Number of Frames"),  # Количество кадров в анимации
        gr.Slider(0.001, 0.1, value=0.02, step=0.001, label="Tree Growth Probability (p)"),  # Вероятность роста дерева
        gr.Slider(0.0, 0.01, value=0.0001, step=0.0001, label="Lightning Probability (f)"),  # Вероятность удара молнии
        gr.Slider(0.01, 1.0, value=0.2, step=0.01, label="Initial Forest Fraction"),  # Начальная доля леса
    ],
    outputs=gr.Gallery(label="Forest Fire Animation Grid (MP4)", columns=5, height="auto"),  # Галерея для вывода видео
    title="Forest Fire Cellular Automaton Animation Grid",  # Заголовок интерфейса
    description="Simulates a GRxGR grid of independent forest fire cellular automata. Download the MP4 animations."
).launch()  # Запуск веб-интерфейса